# DT-BASE
#### Causal Model Execution

In [ ]:
__author__ = 'Andrea Roy (SoTeRiA Research Laboratory)'
__version__ = '1.0'
__email__ = 'akroy2@illinois.edu'

## Introduction
This notebook builds and executes the DT-BASE causal model case study for Training Quality presented in [this](https://github.com/SoTeRiALab/dt-base.git) publication using the updated methodology developed as part of the PSA 2021 conference paper "Managing Unexpected Failures of Nuclear Power Plants by Quantifying Uncertainty of Organizational Impacts on Probabilistic Risk Assessment" (accepted Aug. 2021). 

#### What is DT?
`DT-BASE` is a submodule of the **Data-Theoretic (DT)** approach to quantifying organizational impacts on risk scenarios. DT combines theoretical causal modeling of organizational factors with plant-specific data to provide enhanced realism in the quantification of organizational factors' impacts on risk scenarios, and avoid potentially misleading conclusions which are possible in solely data-oriented approaches.

#### Bayesian Belief Network
`DT-BASE` leverages a Bayesian Belief Network (BBN) to model the extended causality between organizational factors. A BBN is a statistical learning technique which uses a graphical network in the form of a Directed Acyclic Graph (DAG). Each node in the DT-BASE model represents an organizational factor (e.g., training procedure quality, program design quality, etc.). For simplicity, each organizational factor is rated on a scale of `{Good, Bad}` (the "states" of each node). Each edge represents a *causal relationship* between organizational facotrs (e.g., The goal is to quantify `P(Target = Good)`, where `Target` is the "target node," or the "unknown of interest" (e.g., training quality). For every node (organizational factor) in the network, a conditional probability table (CPT) must be developed to model the conditional probability of each child node being in a `Good` or `Poor` stage given the state of a parent node.

#### Importing Evidence

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import references into a pandas dataframe.
refs = pd.read_csv('data/refs.csv')
refs.set_index('ref_id', inplace=True)
refs.head()

In [ ]:
# import nodes into a pandas dataframe
nodes = pd.read_csv('data/nodes.csv')
nodes.set_index('node_id', inplace=True)
nodes.head()

In [ ]:
# import edge data into a pandas dataframe
edges = pd.read_csv('data/edges.csv')
edges.set_index('edge_id', inplace=True)
edges.head()

#### Uncertainty Characterization
There is some element of subjectivity involved in the analyst's estimation of the m1, m2, and m3 values shown, which represent the analyst's judgment of the strength of the causal relationship between parent and child nodes. We model this uncertainty in the analyst's interpretation using an upper and lower bound of the estimate values, rather than simply a point estimate. This allows us to propagate the uncertainty in the analyst's estimates to later stages of the model, using [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) sampling.

In [ ]:
numSamples = 100000

In [ ]:
m1 = pd.DataFrame(columns=refs.index)
m2 = pd.DataFrame(columns=edges.index)
m3 = pd.DataFrame(columns=edges.index)
# generate samples from a uniform distribution for m1, m2, and m3
for ref_id in refs.index:
    m1[ref_id] = np.random.uniform(refs['m1A'][ref_id], refs['m1B'][ref_id], size=(numSamples, ))
    
for edge_id in edges.index:
    m2[edge_id] = np.random.uniform(edges['m2A'][edge_id], edges['m2B'][edge_id], size=(numSamples, ))
    m3[edge_id] = np.random.uniform(edges['m3A'][edge_id], edges['m3B'][edge_id], size=(numSamples, ))

m1.head()

#### Aggregating Edges
The DT-BASE methodology suggests that a minimum of 3 sources of evidence should be collected to support each causal relationship (edge) in the model. As a result, we need a way to aggregate multiple sources of information into a single edge "weight."

In [ ]:
# calculate the "normalized" weight of every edge in the graph using arithmetic mean
normWeights = {}
for edge_id in edges.index:
    edge = (edges.parent_id[edge_id], edges.child_id[edge_id])
    normWeights[edge] = normWeights.get(edge, 0) + m1[edges.ref_id[edge_id]] * m3[edge_id]
    
weights = {}
for edge_id in edges.index:
    edge = (edges.parent_id[edge_id], edges.child_id[edge_id])
    weights[edge_id] = (m1[edges.ref_id[edge_id]] * m3[edge_id]) / normWeights[edge]

In [ ]:
# create superEdges (edges which are the aggregation of multiple edges supporting the same causal relationship)
superEdges = {}
for edge_id in edges.index:
    edge = (edges.parent_id[edge_id], edges.child_id[edge_id])
    superEdges[edge] = superEdges.get(edge, 0) + weights[edge_id] * m2[edge_id]
superEdges

#### DT-BASE BBN

We now have all the information we need to build the DT-BASE BBN. Let's plot a visual representation of the DAG.

In [ ]:
import networkx as nx
from networkx.drawing.nx_pylab import draw_networkx

G = nx.DiGraph()
G.add_edges_from(superEdges)
draw_networkx(G, pos=nx.spring_layout(G), arrowsize=20, node_size=1200, node_color="#DDDDDD")

#### Calculate Conditional Probability Tables (CPTs)
The next step in the execution of the model is to calculate conditional probability tables (CPTs). For this calculation, we use the Noisy-Or approximation. A discussion of this method is beyond the scope of this activity, but if you are interested, we recommend you read [this](https://people.csail.mit.edu/dsontag/papers/HalpernSontag_uai13.pdf) paper for more information.

In [ ]:
import itertools
import pyeda
from pyeda.inter import *

In [ ]:
# develop CPT combinations for each node
cptCombos = {}
for adj in G.reverse().adjacency():
    node = adj[0]
    incoming = list(adj[1].keys())
    bools = []
    tmp = itertools.product([True, False], repeat = len(incoming))
    for t in tmp:
        bools.append(tuple([exprvar(incoming[i]) if t[i] else ~exprvar(incoming[i]) for i in range(len(t))]))
    cptCombos[node] = bools
cptCombos

In [ ]:
# create a leak variable, representing the probability that the model is incomplete.
leak = np.random.uniform(.9, .99, (numSamples, ))

In [ ]:
# calculate the conditional probability table for each node, using the CPT combinations, and the noisy-or approximation.
cpts = { node: [] for node in nodes.index }
for node, combinations in cptCombos.items():
    for combo in combinations:
        tmp = 1
        for parent in combo:
            if isinstance(parent, pyeda.boolalg.expr.Variable):
                tmp *= (1 - superEdges[parent.name, node])
            cpts[node].extend([1 - leak * tmp, leak * tmp])
            
# print out the CPT for each node
{ node: [np.mean(cptVec) for cptVec in cpt] for node, cpt in cpts.items() }

#### BBN Quantification
A BBN model consists of two main features: (1) a DAG representing the structure of the model, and (2) a set of CPTs, one for each node in the model. Now that we have both of these aspects of the BBN model, we can quantify the model. To do this, we will use the `pybbn` python package for Bayesian Inference. We will calculate the `P(tq=Bad)` using the inference algorithm provided by `pybbn`.

In [ ]:
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.pptc.inferencecontroller import InferenceController

In [ ]:
# create BBN nodes
bbnNodes = { node.Index : BbnNode(Variable(i, node.Index, ['Good', 'Bad']), [1, 0]) for i, node in enumerate(nodes.itertuples()) }
bbnNodes

In [ ]:
# create BBN edges
bbnEdges = [ Edge(a, b, EdgeType.DIRECTED) for a, b in superEdges]
bbnEdges

In [ ]:
# construct the DT-BASE BBN graph using the nodes and edges above.
dt_base = Bbn()
for node in bbnNodes.values():
  dt_base.add_node(node)
for edge in bbnEdges:
  dt_base.add_edge

Now, we can run the BBN inference algorithm in the cell below. Please note that this cell may take a couple minutes to execute.

In [ ]:
# perform Bayesian inference for the target node "tq"
# run the model numSamples times to obtain a probability distribution
target = bbnNodes['tq']
target_probs = []

for sample in range(numSamples):
  for node_id, node in bbnNodes.items():
    probs = [p[sample] for p in cpts[node_id]] if cpts[node_id] else [1, 0]
    node.probs = probs
    
  join_tree = InferenceController.apply(dt_base)
  target_probs.append(join_tree.get_bbn_potential(target))

#### Analysis
Now that we have calculated the probability distribution for the target node "Training Quality" being in a "Bad" state, let us display our results and do some simple data analysis.

First, let's output the raw probability distribution

In [ ]:
p = pd.Series([t.entries[1].value for t in target_probs])
p

Now, let's output the summary statistics for the distribution.

In [ ]:
p.describe()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

ax = plt.subplot(111)
ax.hist(p, 100)
ax.set_title('DT-BASE Baseline Quantification')
ax.set_xlabel('P(Training Quality = "Bad")')
ax.set_ylabel('PDF')
plt.style.use('seaborn')
plt.show()